In [1]:

import pandas as pd
soil_data = pd.read_csv("../data/input.csv")
soil_data.head()

,HEADING,LOCA_ID,SCPG_TESN,SCPT_DPTH,SCPT_RES,SCPT_FRES,SCPT_PWP2,SCPT_FRR,SCPT_QT,SCPT_QNET,SCPT_BQ,SCPT_CIC1,SCPT_CISB,SCPT_NQC1,SCPT_NRF1,SCPT_CUW1,FILE_FSET
0,UNIT,NaN,NaN,m,MN/m2,kN/m2,kN/m2,%,MN/m2,MN/m2,NaN,NaN,NaN,NaN,NaN,kN/m3,NaN
1,TYPE,ID,X,2DP,3DP,3DP,1DP,3DP,3DP,3DP,4DP,2DP,2DP,1DP,3DP,2DP,X
2,DATA,HKN14-BH-SA,CPT01,26.50,0.650,NaN,NaN,NaN,0.718,0.196,0.0374,NaN,NaN,NaN,NaN,NaN,NaN
3,DATA,HKN14-BH-SA,CPT01,26.52,1.691,NaN,265.1,NaN,1.758,1.235,0.0011,NaN,NaN,NaN,NaN,NaN,NaN
4,DATA,HKN14-BH-SA,CPT01,26.54,2.625,NaN,262.2,NaN,2.690,2.167,-0.0015,NaN,NaN,NaN,NaN,NaN,NaN
